<a href="https://colab.research.google.com/github/JaxsonStentz/AI_Toolkit/blob/main/AI_Toolkit_Website_Brochure_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip installs for Google Colab

!pip install python-dotenv
!pip install httpx==0.27.2
!pip install openai==1.55.3

In [2]:
# Imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Load environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [4]:
# Create a class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
# Individual website scraper (get the content from the webpage)

ed = Website("https://walmart.com")
print(ed.get_contents())

Webpage Title:
Walmart | Save Money. Live better.
Webpage Contents:
Skip to Main Content
How do you want your items?
Cancel
Reorder
My Items
Reorder
Lists
Registries
Sign In
Account
Sign in or create account
Purchase History
Walmart+
$0.00
Departments
Services
|
How do you want your items?
Walmart | Save Money. Live better.
Presents for them, too!
Shop pets
Shop pets
Great finds for techies
Shop gifts
Shop gifts
Styles that wow
Shop gifts
Shop gifts
Calling all Santas!
Save on 1,000s of gifts
Shop all
Shop all
Gift-swap goodies
Shop now
Shop now
Under
$
25
Up to 65% off
Shop now
Shop now
Join Walmart+
Walmart+ delivers fast & free from your local store
One free trial per member. Terms apply.
Join Walmart+
Make it personal, from $10
Shop gifts
Shop gifts
Surprise the host
Shop gifts
Shop gifts
Buzzy faves to give & get
Shop gifts
Shop gifts
Starting at
$
15
We’d love to hear what you think!
Give feedback
All Departments
Store Directory
Careers
Our Company
Sell on Walmart.com
Help
Produc

In [6]:
# System prompt

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
# Define links_user_prompt

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
# Individual website scraper (get the links from the webpage only)

print(get_links_user_prompt(ed))

Here is the list of links on the website of https://walmart.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#maincontent
/all-departments
/
/
/my-items
/my-items
/lists
/my-registries
/
/orders
/plus
/shop/gift-finder?facet=gf_occasion%3AChristmas%7C%7Cgf_interests%3APets&athAsset=eyJhdGhjcGlkIjoiNGY4MTQxYjYtMzA1OC00ZDYxLTlkM2EtYjkwMGZkYjFkNjk1In0=&athena=true
/shop/gift-finder?facet=gf_occasion%3AChristmas%7C%7Cgf_interests%3APets&athAsset=eyJhdGhjcGlkIjoiNGY4MTQxYjYtMzA1OC00ZDYxLTlkM2EtYjkwMGZkYjFkNjk1In0=&athena=true
/shop/gift-finder?facet=gf_occasion%3AChristmas%7C%7Cgf_interests%3ATech&athAsset=eyJhdGhjcGlkIjoiMTVlYjZjNWQtZTllYy00ODM4LWE4NjItYTIxODMzMDFiMGJmIn0=&athena=true
/shop/gift-finder?facet=gf_occasion%3AChristmas%7C%7Cgf_interests%3ATech&athAsset=eyJhdGhjcGlkIjoiMTVlYjZjNWQtZTllYy00ODM4LWE4

In [9]:
# Define get_links

def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [10]:
# Pull important links from list of links from link webscraper

get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://anthropic.com/about'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'},
  {'type': 'company page', 'url': 'https://anthropic.com/company'}]}

In [11]:
# Define get_all_details

def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [12]:
# Print all details from website

print(get_all_details("https://walmart.com"))

Found links: {'links': [{'type': 'careers page', 'url': 'https://careers.walmart.com/'}, {'type': 'corporate page', 'url': 'https://corporate.walmart.com/'}]}
Landing page:
Webpage Title:
Walmart | Save Money. Live better.
Webpage Contents:
Skip to Main Content
How do you want your items?
Cancel
Reorder
My Items
Reorder
Lists
Registries
Sign In
Account
Sign in or create account
Purchase History
Walmart+
$0.00
Departments
Services
|
How do you want your items?
Walmart | Save Money. Live better.
Presents for them, too!
Shop pets
Shop pets
Great finds for techies
Shop gifts
Shop gifts
Styles that wow
Shop gifts
Shop gifts
Calling all Santas!
Save on 1,000s of gifts
Shop all
Shop all
Gift-swap goodies
Shop now
Shop now
Under
$
25
Up to 65% off
Shop now
Shop now
Join Walmart+
Walmart+ delivers fast & free from your local store
One free trial per member. Terms apply.
Join Walmart+
Make it personal, from $10
Shop gifts
Shop gifts
Surprise the host
Shop gifts
Shop gifts
Buzzy faves to give & g

In [13]:
# System prompt

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure:

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [14]:
# Define brochure_user_promp

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [15]:
# Define create_brochure

def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [16]:
# Create the brochure of the website!

create_brochure("Walmart", "https://walmart.com")

Found links: {'links': [{'type': 'careers page', 'url': 'https://careers.walmart.com/'}, {'type': 'company page', 'url': 'https://corporate.walmart.com/'}]}


```markdown
# Walmart Brochure

## About Us
At Walmart, our mission is clear: **"Save Money. Live Better."** We are dedicated to providing our customers with affordable products, a diverse selection, and a commitment to sustainability. With over 2.2 million associates worldwide, Walmart stands as a leader in retail, ensuring that everyone, from the everyday shopper to businesses, has access to what they need at prices they can afford.

---

## Our Culture
Our culture embodies our values in action. At Walmart, we strive to create a workplace where diversity, equity, and inclusion are not just ideals but realities. We understand that our associates bring unique perspectives and experiences that are essential for innovation and success.

### Diversity & Inclusion
We believe that fostering a workplace culture where everyone is included enhances our performance and drives growth. Our commitment to diversity is evident in our hiring practices and community engagements.

### Employee Benefits
- **Live Better U**: A unique education benefits program allowing associates to further their education, fully funded by Walmart.
- **Smart Benefits**: A comprehensive package tailored to help our associates thrive both personally and professionally.
- **Competitive Pay**: Store managers have the potential to earn an annual bonus up to 200% of their base salary, signifying our investment in leadership.

---

## Career Opportunities
Walmart offers diverse career paths across various sectors, including:

- **Retail Management**
- **Distribution & Fulfillment**
- **Healthcare**
- **Technology & Cybersecurity**
- **Corporate Services**

Whether you're looking to start your career or take it to the next level, Walmart has opportunities for individuals at every stage, inclusive of students and military veterans.

---

## Our Customers
We serve millions of customers every day across our stores and online platforms. From budget-conscious shoppers to business clients, we cater to a wide array of needs, ensuring that quality and savings go hand in hand.

## Sustainability Commitment
Walmart is dedicated to making a positive impact on the environment. Through responsible sourcing and sustainability initiatives, we work to make the communities we serve better places to live and work.

---

## Join Us
Are you ready to be part of a team that values innovation, service, and community? **Join Walmart today and help us drive the next great retail disruption!**

For more information on careers, culture, and our commitment to our customers and communities, visit us at [Walmart Careers](https://www.walmart.com/careers).

---

*Together, we can not only save money but enhance lives and communities. At Walmart, we are not just about transactions; we are about transformation.*
```


In [17]:
# Define stream_brochure

def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [19]:
# Stream the brochure of the website!

stream_brochure("Walmart", "https://walmart.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://corporate.walmart.com/'}, {'type': 'careers page', 'url': 'https://careers.walmart.com/'}]}


# Walmart Company Brochure

## **Walmart - Save Money. Live Better.**

### **Overview**
Walmart is a leading global retailer dedicated to providing customers with low prices, a wide range of products, and excellent service. With a mission to help families save money and live better, Walmart offers everything from groceries to electronics, making it a one-stop-shop for all your needs.

### **Company Culture**
At Walmart, we define our culture as our values in action. Our commitment to diversity, equity, and inclusion is paramount; we strive to create a workplace that values unique experiences and perspectives. Initiatives like **Live Better U**, which offers educational benefits for associates, showcase our commitment to investing in our employees' futures. 

Walmart fosters a culture that encourages innovation and the pursuit of excellence—whether that be through technology or customer service. Our corporate culture is built on integrity, respect, and a will to succeed together, creating an environment where everyone can thrive.

### **Customers**
Walmart serves millions of customers every week across the globe. With a wide range of departments, including Grocery, Electronics, Clothing, Health & Wellness, and more, we cater to diverse needs and preferences. Whether shopping in-store or online, our customers can enjoy savings every day.

### **Careers at Walmart**
Walmart is continuously seeking committed individuals who are ready to contribute to our mission. Our career opportunities span across various areas, including:

- **Retail Management**
- **Distribution and Fulfillment**
- **Technology and Cybersecurity**
- **Healthcare Services**
- **Corporate Functions**

We prioritize career development and offer clear paths for growth. Employees can benefit from competitive pay, health benefits, retirement plans, and a comprehensive education support program. Join us, and be part of a team that is revolutionizing retail.

### **Impact and Community Engagement**
Walmart is dedicated to making a positive impact in the communities we serve. Through sustainability initiatives, charitable contributions, and programs that incentivize local sourcing and manufacturing, we are committed to creating a better future for our associates, customers, and communities.

### **Join Us**
If you're ready to make a difference and be part of a team that values your contributions, explore the opportunities available at Walmart today. Together, we can save money and live better!

---

For more information, visit our website at [Walmart Careers](https://www.walmart.com/careers) and learn more about our commitment to our customers, associates, and communities.